In [2]:
# Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt

# Import the SQL database into Pandas.
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Create Database Connection
import db_key

engine = create_engine('postgresql://{username}{password}@localhost:{port}/Employee_db')
connection = engine.connect()


ValueError: invalid literal for int() with base 10: '{port}'

In [ ]:
# Create Classes
class department(Base):
    __tablename__ = 'departments'
    dept_no = Column(String)
    dept_name = Column(String)
    
class dept_emp(Base):
    __tablename__ = 'dept_emp'
    emp_no = Column(Integer)
    dept_no = Column(String)
    
class dept_manager(Base):
    __tablename__ = 'dept_manager'
    dept_no = Column(String)
    emp_no = Column(Integer)
    
class employees(Base):
    __tablename__ = 'employees'
    emp_no = Column(Integer)
    emp_title_id = Column(String)
    birth_date = Column(Date)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(String)
    hire_date = Column(Date)
    
class salaries(Base):
    __tablename__ = 'salaries'
    emp_no = Column(Integer)
    salary = Column(Integer)
    
class titles(Base):
    __tablename__ = 'titles'
    title_id = Column(String)
    title = Column(String)

In [ ]:
# Create a histogram to visualize the most common salary ranges for employees.
#salaries.plot(kind='hist')
plt.hist()

In [ ]:
# Create a bar chart of average salary by title.
plt.bar()